In [1]:
import pandas as pd

data = pd.read_parquet("../dump/processed/data.parquet")


print(data.iloc[0]["text"])

April (Apr.) is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days. April always begins on the same day of the week as July, and additionally, January in leap years. April always ends on the same day of the week as December. == The Month == April comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year. April begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart. In common years, April starts on the same day of the week as October of the previous year, and in leap years, May of the previous year. In common years, April finishes on the same day of the week as July of the previou

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from os import getenv
from dotenv import load_dotenv
import concurrent.futures
import time

# Load environment variables
load_dotenv()

def format_text_to_markdown(raw_text: str) -> str:
    """
    Converts raw text into clean, properly formatted markdown.

    Args:
        raw_text (str): The input text to be formatted.

    Returns:
        str: The formatted markdown text.
    """
    template = """Convert the following raw text into clean, properly formatted markdown. Apply appropriate:
    - Headings
    - Bullet points
    - Code blocks (if applicable)
    - Emphasis
    - Spacing and line breaks

    Text:

    {text}
    """

    prompt = PromptTemplate(template=template, input_variables=["text"])

    llm = ChatOpenAI(
        openai_api_key=getenv("OPENROUTER_API_KEY"),
        openai_api_base="https://openrouter.ai/api/v1",
        model_name="google/gemini-2.0-flash-exp:free",
    )

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser

    return chain.invoke(input={"text": raw_text})

def parallel_format_texts(texts, max_workers=None):
    """
    Formats multiple texts in parallel using the format_text_to_markdown function.

    Args:
        texts (list of str): List of raw texts to be formatted.
        max_workers (int, optional): Maximum number of threads to use. If None, uses the default (maximum possible).

    Returns:
        list of str: List of formatted markdown texts.
    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(format_text_to_markdown, text) for text in texts]
        formatted_texts = [future.result() for future in concurrent.futures.as_completed(futures)]
    return formatted_texts

In [3]:
texts = [
    data.iloc[0]["text"],
    data.iloc[1]["text"],
    data.iloc[2]["text"]
]

# Case 1: Use the default number of threads
start_time = time.time()
formatted_texts_default = parallel_format_texts(texts)
end_time = time.time()
print(f"Total execution time (default threads): {end_time - start_time:.2f} seconds")

/var/folders/47/gybx85kd6f147h3rp8wgdz9w0000gn/T/ipykernel_99621/3695377915.py:36: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/47/gybx85kd6f147h3rp8wgdz9w0000gn/T/ipykernel_99621/3695377915.py:36: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


TypeError: 'NoneType' object is not iterable